In [3]:
!pip install langchain langchain-core langchain-community langchain_experimental duckduckgo-search pydantic

  Using cached langchain_community-0.3.26-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_experimental-0.3.4-py3-none-any.whl.metadata (1.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


## DuckDuck-Go Search

In [5]:
from langchain_community.tools import DuckDuckGoSearchRun
search=DuckDuckGoSearchRun()
result=search.invoke("What is the news in pakistan today")

In [6]:
print(result)

PESHAWAR, Pakistan -- A suicide bombing in northwest Pakistan on Saturday killed at least eight soldiers and injured 25 people, including civilians, officials said. The attack targeted a military ... Stay on top of Pakistan latest developments on the ground with Al Jazeera's fact-based news, exclusive video footage, photos and updated maps. PakMirror.com is your ultimate destination for the latest news, trending stories, and in-depth coverage of Pakistan's current affairs. From politics and business to technology and sports, we bring you fresh updates and expert insights, keeping you informed and ahead of the curve. Pakistan and the U.S. have resolved to conclude trade talks next week, the South Asian nation said on Wednesday after a meeting between its Finance Minister Muhammad Aurangzeb and U.S. Commerce ... Stay current with all the latest and breaking news about Pakistan, compare headlines and perspectives between news sources on stories happening today.In total, 16,797stories have

In [7]:
print(search.name)
print(search.description)
print(search.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


## Custom Tools using Tool Decorator

In [8]:
from langchain.tools import tool

In [9]:
# Step 1 create a function
def multiply(a,b):
  return a * b

In [12]:
## step 2 add type hints and comments
def multiply(a:int,b:int) ->int:
  """Multiply Two Numbers"""
  return a * b

In [13]:
## Step 3 Add a tool decorator

@tool
def multiply(a:int,b:int)->int:
  "Multiply two numbers"
  return a * b

In [14]:
result=multiply.invoke({'a':3,'b':5})

In [15]:
print(result)

15


In [16]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [17]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


## Custom Tools using Structured Tool

In [21]:
from langchain.tools import StructuredTool
from pydantic import BaseModel,Field


In [23]:
from typing_extensions import Required
class MultiplyInput(BaseModel):
  a: int=Field(Required=True,description="The First NUmber to Multiply")
  b: int=Field(Required=True, description="The Second Number to Multiply")

In [24]:
## step 2 add type hints and comments
def multiply(a:int,b:int) ->int:
  return a * b

In [28]:
multiply_tool=StructuredTool.from_function(
    func=multiply,
    name="Multiply",
    description="Multiply two numbers",
    arguments_schema=MultiplyInput
)

In [29]:
result=multiply_tool.invoke({'a':3,'b':4})

In [30]:
print(result)

12


## Custom Tool Using BaseTool Class

In [31]:
from langchain.tools import BaseTool
from typing import Type

In [33]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to multiply")
    b: int = Field(required=True, description="The second number to multiply")

In [34]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [35]:
multiply_tool = MultiplyTool()

In [36]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to multiply', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to multiply', 'required': True, 'title': 'B', 'type': 'integer'}}


## Toolkit

In [37]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


In [38]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]


In [39]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)


add => Add two numbers
multiply => Multiply two numbers
